In [68]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

In [3]:
#FUNCIÓN PARA TRADUCIR

def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated

#FUNCIÓN PARA LIMPIAR

def cleaning(text):
    text = text.lower()
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    return doc



In [4]:
# LEER DATAFRAME Y EXTRAER JOB HISTORY

df = pd.read_excel("2_final_db.xlsx")
# df = df [["id", "job_history"]]
df = df[df.job_history.notna()]

In [5]:
#SEPARANDO LOS STRINGS DE CADA TRABAJO

L = []

for i in df.job_history:
    l = str(i).split("},") #convertir en string el job history del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for job in l: #para cada trabajo
        job = job.replace("[", "").replace("]", "") + "}" #separelos y conviertalas en diccionarios
        k.append(job) #añadalas a la lista de jobs
    L.append(k) 

In [14]:
#EXTRAER DE MANERA ORDENADA EL JOB HISTORY, LA FECHA DE TERMINACIÓN Y COMIENZO

K = [] 

for individuo in tqdm(L): 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        #EXTRAER JOB-TITLE
        if ( len(re.findall("'job_title': '.*?'", ins)) ) > 0:
            c["job_title"] =  re.findall("'job_title': '.*?'", ins)[0].replace("'job_title': ", "").replace("'", "")
            c["job_title"] = str(c["job_title"])
            # c["job_title"] = cleaning(c["job_title"])

        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = int(c["start"][::-1][0:4][::-1]) #SOLO CONSERVAR EL AÑO 

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = int(c["end"][::-1][0:4][::-1]) #SOLO CONSERVAR EL AÑO

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

100%|██████████| 389/389 [00:00<00:00, 9046.72it/s]


In [20]:
#TRADUCIR Y LIMPIAR 

for i in tqdm(K[:]):
    for e in tqdm(i):
        # A = "job_title" in e
        # if len(e["job_title"])>0 and A == True:
        try:
            e["job_title"] = translateString(str(e["job_title"]), "en")
            e["job_title"] = cleaning(e["job_title"])
        except:
            None
        

100%|██████████| 389/389 [38:11<00:00,  5.89s/it]


In [22]:
#CREAR LA COLUMNA EN EL DATAFRAME
df["job_history_clean"] = K

In [27]:
df.to_excel("final_db_jh_clean.xlsx", index = False)

In [97]:
#EXTRACCIÓN DEL AÑO DE GRADUACIÓN

In [204]:
df = pd.read_excel("final_db_jh_clean.xlsx")
db = pd.read_excel("2_final_db.xlsx")

In [205]:
#BUSCAR EN EDUCACIÓN
dfe = db[(db["search"]=="educ_regex") | (db["search"]=="educ_fp") | (db["search"]=="educ_fs")]
dfe["education"] =  dfe["education"].str.lower()

<ipython-input-205-a6fafd69d3af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["education"] =  dfe["education"].str.lower()


In [206]:

L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]


K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = int(c["start"])

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = int(c["end"])

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

dfe["educacion"] = K


<ipython-input-206-95037ae4090b>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["educacion"] = K


In [207]:
dfe = dfe.reset_index(drop = True)

dfe["end_boot"] = np.nan 

c = 0

for i in range(len(dfe.educacion)):
    for e in range(len(dfe.educacion[i])): #DENTRO DE LA EDUCACIÓN DE CADA INDIVIDUO
        if "education" in dfe.educacion[i][e].keys() and dfe.educacion[i][e]["education"] == dfe.match[i]:
            if "end" in dfe.educacion[i][e].keys():
                dfe["end_boot"][i] = dfe.educacion[i][e]["end"]
            elif "start" in dfe.educacion[i][e].keys():
                dfe["end_boot"][i] = dfe.educacion[i][e]["start"] #SI NO TENÍA EL FINAL POR LO MENOS SE LE IMPUTÓ EL INICIAL, EXISTE LA POSIBILIDAD DE QUE NO LO HAYA TERMINADO


dfe["end_boot"][0]

<ipython-input-207-77869553395e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["end_boot"][i] = dfe.educacion[i][e]["end"]
<ipython-input-207-77869553395e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe["end_boot"][i] = dfe.educacion[i][e]["start"] #SI NO TENÍA EL FINAL POR LO MENOS SE LE IMPUTÓ EL INICIAL, EXISTE LA POSIBILIDAD DE QUE NO LO HAYA TERMINADO


2015.0

In [208]:
#CERTIFICACIÓN

dfc = db[(db["search"]=="cert_regex") | (db["search"]=="cert_fp") | (db["search"]=="cert_fs")]
dfc["education"] =  dfc["education"].str.lower()
#CONVERTIR EN STRING
dfc["certifications"] =  dfc["certifications"].str.lower()

#EXTRAER COLUMNA

L = []

for i in dfc["certifications"]:    
    l = str(i).split("},") 
    k = [] 
    for formacion in l: 
        formacion = formacion.replace("[", "").replace("]", "") + "}" 
        k.append(formacion) 
    L.append(k)
     
L[0][0]



K = [] 

dfc = dfc.reset_index(drop = True)

regxCerAuth = "'certificate_authority': '.*?'"
regexTitle = "'title': '.*?'"

for individuo in L: 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        if ( len(re.findall("'certificate_authority': '.*?'", ins)) ) > 0:
            c["certificate_authority"] =  re.findall("'certificate_authority': '.*?'", ins)[0].replace("'certificate_authority': ", "").replace("'", "")

        if ( len(re.findall(regexTitle, ins)) ) > 0:
            c["title"] =  re.findall("'title': '.*?'", ins)[0].replace("'title': ", "").replace("'", "")

        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            try:
                c["start"] = int(c["start"][::-1][0:4][::-1])
            except:
                None

        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            try:
                c["end"] = int(c["end"][::-1][0:4][::-1])
            except:
                None

        educacion_ind.append(c) 
    
    K.append(educacion_ind) 

K[0][0]

dfc["certifications"] = K



<ipython-input-208-658f21cfa4ac>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["education"] =  dfc["education"].str.lower()
<ipython-input-208-658f21cfa4ac>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["certifications"] =  dfc["certifications"].str.lower()


In [209]:
dfc["certifications"][0] #HAY GENTE QUE SIGUE HACIENDOLO O NO LO TERMINÓ EN ESE CASO NO SE LE CONVIRTIÓ EN STRING

[{'certificate_authority': 'agbo',
  'title': 'course of basics ios programming (agbo training)',
  'start': 2013,
  'end': 2013},
 {'certificate_authority': 'agbo',
  'title': 'course of advanced ios programming',
  'start': 2014,
  'end': 2015},
 {'certificate_authority': 'keepcoding',
  'title': 'swift for objective-c developers',
  'start': 2015,
  'end': 'present'}]

In [210]:
dfc["end_boot"] = np.nan 

c = 0

for i in range(len(dfc.certifications)):
    for e in range(len(dfc.certifications[i])): #DENTRO DE LA EDUCACIÓN DE CADA INDIVIDUO
        if dfc.cert_field_match[i] == "authority" or dfc.cert_field_match[i] == "certificate_authority":
                if "certificate_authority" in dfc.certifications[i][e] and dfc.certifications[i][e]["certificate_authority"] == dfc.certificate_authority[i]:
                    if "end" in dfc.certifications[i][e].keys() and dfc.certifications[i][e]["end"]!="present": #DEJÉ AÑO EN QUE EMPEZÓ

                    # if "end" in dfc.certifications[i][e].keys():
                        dfc["end_boot"][i] = dfc.certifications[i][e]["end"]

                    elif "start" in dfc.certifications[i][e].keys():
                        dfc["end_boot"][i] = dfc.certifications[i][e]["start"]

                    else:
                        dfc["end_boot"][i] = dfc.certifications[i][e]["end"]

        if dfc.cert_field_match[i] == "title":
            if "title" in dfc.certifications[i][e] and dfc.certifications[i][e]["title"] == dfc.title[i]:
            
                    if "end" in dfc.certifications[i][e].keys() and dfc.certifications[i][e]["end"]!="present": #DEJÉ AÑO EN QUE EMPEZÓ
                    # if "end" in dfc.certifications[i][e].keys():
                        dfc["end_boot"][i] = dfc.certifications[i][e]["end"]

                    elif "start" in dfc.certifications[i][e].keys():
                        dfc["end_boot"][i] = dfc.certifications[i][e]["start"]   

                    else:
                        dfc["end_boot"][i] = dfc.certifications[i][e]["end"] #en caso de que no tenga inicio y en el final tenga presente, como pasa con el individuo 4



<ipython-input-210-9078b050ebfa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["end_boot"][i] = dfc.certifications[i][e]["start"]
<ipython-input-210-9078b050ebfa>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["end_boot"][i] = dfc.certifications[i][e]["end"]
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipyth

In [211]:
final_año = dfe.append(dfc)
final_año = final_año[["id", "end_boot"]]

In [212]:
df = pd.merge(df, final_año, "left", "id")
df = df.to_excel("final_db_jh_clean_endboot.xlsx", index = False)